# **TIỀN XỬ LÝ** 

## Các bước thực hiện:
 - Kiểm tra và tiền xử lý output 
 - Tách dữ liệu thành 3 tập: train, validaton, test.
 - Tiến hành tiền xử lý


In [1]:
# IMPORT THƯ VIỆN
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

## Khám phá dữ liệu (đủ để có thể xác định câu hỏi)

In [2]:
data = pd.read_csv('HOUSEs_info.csv', sep = '\t')
data

,Loại hình,Phòng ngủ,Phòng tắm,Diện tích,Tình hình nội thất,Dự án,Tháp,Giá bán,Ban công,Phòng cho giúp việc,...,Vườn tiểu cảnh,khuôn viên đi dạo,Khu vui chơi trẻ em,Giếng trời,Gác Lửng,Quạt trần hàng Ngoại,Hồ cá,3 máy lạnh,1 phòng thờ,1 sân phơi
0,Căn hộ,3.0,2.0,83.15 m²,Đầy đủ nội thất,Sunrise Riverside,G1,4.19 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Căn hộ,1.0,1.0,53.2 m²,Nội thất cơ bản,Q7 Saigon Riverside,MERCURY,1.87 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Căn hộ,3.0,2.0,105.8 m²,Nội thất cơ bản,Vinhomes Central Park,NaN,6.5 tỷ,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Căn hộ,2.0,2.0,106.1 m²,Nội thất cơ bản,Sunrise City,NaN,4.25 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Căn hộ,2.0,1.0,60 m²,Đầy đủ nội thất,Vinhomes Grand Park,S2.05,2.65 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7149,Căn hộ,1.0,1.0,52.13 m²,Nội thất cơ bản,City Gate 3,NaN,1.43 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7150,Căn hộ,1.0,1.0,50 m²,Nội thất cơ bản,The Metropole Thủ Thiêm,NaN,7.8 tỷ,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7151,Căn hộ,2.0,1.0,59 m²,Nội thất cơ bản,Vinhomes Grand Park,S3.01,2.53 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7152,Căn hộ,2.0,2.0,70.5 m²,Nội thất cơ bản,Precia,NaN,3.95 tỷ,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
data.shape


(7154, 120)

### Dữ liệu có các dòng bị lặp không?

In [4]:
data.index.duplicated().sum()

0

## Đưa ra câu hỏi cần trả lời
*Output - Giá bán -* được tính từ *input - các thông tin của hành khách -* theo công thức nào?
 Trong đồ án này, ý nghĩa thực tế của việc đi tìm câu trả lời cho câu hỏi trên giúp người dùng có được 1 sự tham khảo với 1 độ chính xác cụ thể về giá nhà dựa trên các thông tin. Qua đó, người dùng có thể dựa trên dự đoán để tự đưa ra quyết định về việc mua (bán) nhà với một mức giá hợp lý.  


## Khám phá dữ liệu 
### 1. OUTPUT TRƯỚC KHI TIỀN XỬ LÝ

In [5]:
# Cột output hiện có kiểu dữ liệu gì?
data['Giá bán'].dtype

dtype('O')

In [6]:
# Cột output có giá trị thiếu không?
data['Giá bán'].isna().sum()

396

#### TIỀN XỬ LÝ OUTPUT

In [7]:
# Xoá ouput có giá trị thiếu
data = data[data['Giá bán'].notna()]
data['Giá bán'].isna().sum()
data = data.reset_index(drop = True) 

In [8]:
# Chuẩn hoá về dạng số

# Vị trí các số hảng tỷ
mil_index = [] 
# Vị trí các số hảng triệu
bil_index = []

for i in range(0, data.shape[0]):
    value = data['Giá bán'][i] 
    if 'tỷ' in value:
        data.at[i, 'Giá bán'] = value.rstrip('tỷ')
        bil_index.append(i)
    elif 'tr' in value:
        data.at[i, 'Giá bán'] = value.rstrip(' tr')
        mil_index.append(i)

In [9]:
# Chuyển đơn vị triệu thành tỷ (đồng)
data['Giá bán'] = data['Giá bán'].astype(float)
for i in mil_index:
    data.at[i, 'Giá bán'] = data['Giá bán'][i] / 1000

### 2. SAU KHI TIỀN XỬ LÝ OUTPUT

In [10]:
# Cột output hiện có kiểu dữ liệu gì?
data['Giá bán'].dtype

dtype('float64')

In [11]:
# Cột output có giá trị thiếu không?
data['Giá bán'].isna().sum()

0

## Tiền xử lý (tách các tập)
    Ta tách các tập từ dữ liệu thu thập được thành 3 tập: train, validation và test theo tỉ lệ 70%:15%:15%

In [12]:
# Tách X và y
y_sr = data['Giá bán'] # sr là viết tắt của series
X_df = data.drop("Giá bán", axis=1)

In [13]:
# Tách theo tỉ lệ: 7:2:1
# Tách dữ liệu thu thập thành 2 tập: tập huấn luyện và tập other (validation và test) theo tỉ lệ 60%:40%
train_X_df, other_X_df, train_y_sr, other_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

# Tách tập other (validation và test) thành 2 tập: tập validation và tập test theo tỉ lệ 50%:50% (tức: bằng 15%:15% so với dữ liệu ban đầu)
val_X_df, test_X_df, val_y_sr, test_y_sr = train_test_split(other_X_df, other_y_sr, test_size=0.5, random_state=0)

train_X_df.shape, val_X_df.shape, test_X_df.shape, train_y_sr.shape, val_y_sr.shape, test_y_sr.shape

((4730, 119), (1014, 119), (1014, 119), (4730,), (1014,), (1014,))

## Khám phá dữ liệu (tập huấn luyện)

In [14]:
train_X_df.dtypes

Loại hình                object
Phòng ngủ               float64
Phòng tắm               float64
Diện tích                object
Tình hình nội thất       object
                         ...   
Quạt trần hàng Ngoại     object
Hồ cá                    object
3 máy lạnh               object
1 phòng thờ              object
1 sân phơi               object
Length: 119, dtype: object

In [15]:
train_X_df.dtypes[train_X_df.dtypes != object]

Phòng ngủ    float64
Phòng tắm    float64
Số tầng      float64
dtype: object

In [16]:
dropped_col = []

In [17]:


# Các cột dạng số
num_cols = ['Phòng ngủ', 'Phòng tắm', 'Số tầng']
df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Phòng ngủ,Phòng tắm,Số tầng
missing_ratio,2.0,2.4,47.6
min,0.0,1.0,1.0
lower_quartile,2.0,1.0,3.0
median,2.0,2.0,9.0
upper_quartile,2.0,2.0,17.0
max,31.0,32.0,44.0


In [18]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn

cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])



,Khóa cửa thẻ từ,Tủ bếp dưới\n,Chiều dài,Thời gian bắt đầu bán,Diện tích sàn,Chủ đầu tư Singapore Thảo Điền,Số phòng tắm,Rèm,Chỗ đậu xe hơi,Gác Lửng,...,Quạt trần hàng Ngoại,Bộ bàn ăn và ghế,Tủ treo đồ / tủ quần áo,Máy giặt,Giá đăng,Loại hình,Loại chủ quyền,Máy sấy,Nhà trống,Mã nhà đất
missing_ratio,0,99.7,98.8,85.5,99.3,100,100,100,0,100,...,100,2,2,0,100,0,28.5,0,100,100
num_values,2,1,23,396,32,1,0,1,2,0,...,0,2,2,2,0,14,4,2,1,0
value_ratios,"{'No': 99.1, 'Yes': 0.9}",{'No': 100.0},"{'20 m': 25.0, '17 m': 10.7, '19 m': 7.1, '18 m': 5.4, '13 m': 5.4, '14 m': 5.4, '15 m': 5.4, '25 m': 3.6, '12 m': 3.6, '27 m': 3.6, '40 m': 3.6, '60 m': 1.8, '22 m': 1.8, '33 m': 1.8, '30 m': 1.8...","{'28/09/2020': 1.9, '07/12/2020': 1.5, '16/12/2020': 1.5, '26/11/2020': 1.3, '18/12/2020': 1.3, '24/10/2017': 1.0, '27/12/2021': 1.0, '08/12/2020': 1.0, '25/11/2020': 1.0, '08/02/2017': 0.9, '26/1...","{'160 m²': 8.8, '71 m²': 2.9, '200.9 m²': 2.9, '303 m²': 2.9, '72 m²': 2.9, '326.3 m²': 2.9, '170 m²': 2.9, '519 m²': 2.9, '356 m²': 2.9, '167.6 m²': 2.9, '167 m²': 2.9, '400 m²': 2.9, '140 m²': 2...",{'Yes': 100.0},{},{'Yes': 100.0},"{'No': 88.9, 'Yes': 11.1}",{},...,{},"{'No': 92.3, 'Yes': 7.7}","{'No': 91.1, 'Yes': 8.9}","{'No': 98.7, 'Yes': 1.3}",{},"{'Căn hộ': 80.6, 'Nhà phố': 12.3, 'Office-tel': 3.7, 'Đất nền': 0.8, 'Shophouse': 0.8, 'Biệt thự': 0.6, 'Penthouse': 0.5, 'Văn phòng': 0.4, 'Căn hộ dịch vụ': 0.1, 'Loại khác': 0.1, 'Lofthouse': 0....","{'HĐ mua bán': 72.6, 'Sổ hồng': 25.4, 'Sổ đỏ': 2.0, 'Giấy tay': 0.0}","{'No': 99.7, 'Yes': 0.3}",{'Yes': 100.0},{}


In [19]:
for i in range(0, df.shape[1]):
    if df.agg(missing_ratio)[i] >= 50:
#         print(df.columns.values[i], df.agg(missing_ratio)[i])
        dropped_col.append(df.columns.values[i])
len(dropped_col)

56